In [ ]:
import sys
# setting path
sys.path.append('../')

import os
import matplotlib.pyplot as plt
import numpy as np

#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

import torch
torch.cuda.empty_cache()
from models.DVBCDModel import DVBCDModel
from utils.datasets import CamelyonDataset, WSSBDatasetTest
from utils.utils_BCD import undo_normalization, od2rgb_np, C_to_OD_torch, normalize_to1

In [ ]:
print(torch.__version__)
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
torch.autograd.set_detect_anomaly(True)

USE_GPU = True
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
if USE_GPU and torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('using device:', DEVICE)

SAVE_MODEL_PATH = "/work/work_fran/Deep_Var_BCD/weights/1pe_224ps_0.75theta_0.05sigmaRui_60000nsamples/"
SAVE_FIG_PATH = "/work/work_fran/Deep_Var_BCD/results/img/"

In [ ]:
model = DVBCDModel(device = DEVICE)
model.load(SAVE_MODEL_PATH + "best.pt")

# Camelyon

In [ ]:
dataset = CamelyonDataset(data_path = "/data/BasesDeDatos/Camelyon/Camelyon17/training/patches_224/", centers = [0], patch_size = 224, n_samples = 100)
#dataset = WSSBDatasetTest("/data/BasesDeDatos/Alsubaie/Data/", organ_list=["Colon"])

In [ ]:
idx = 3
img, od_img, mR = dataset[idx]
#img, od_img, mR, C_gt, M_gt = dataset[idx]

In [ ]:
od_img = od_img.to(DEVICE)
out_Mnet_mean, out_Mnet_var, out_Cnet, Y_rec = model.forward(od_img.unsqueeze(0))

In [ ]:
img_np = img.numpy().astype(np.uint).transpose(1,2,0)

Y_rec_np =  Y_rec.cpu().detach().numpy().squeeze()
Y_rec_np = od2rgb_np(undo_normalization(Y_rec_np))
Y_rec_np = np.clip(Y_rec_np, 0, 255).astype(np.uint8)
Y_rec_np = Y_rec_np.transpose(1,2,0)

In [ ]:
C_OD = C_to_OD_torch(out_Cnet, out_Mnet_mean).cpu().detach().numpy().squeeze()
H_OD = C_OD[0, :, :, :]
H_RGB = od2rgb_np(undo_normalization(H_OD))
H_RGB = np.clip(H_RGB, 0, 255).astype(np.uint8)
H_RGB = H_RGB.transpose(1,2,0)

E_OD = C_OD[1, :, :, :]
E_RGB = od2rgb_np(undo_normalization(E_OD))
E_RGB = np.clip(E_RGB, 0, 255).astype(np.uint8)
E_RGB = E_RGB.transpose(1,2,0)

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(2,2,1)
plt.imshow(img_np)
plt.title("Original")
plt.subplot(2,2,2)
plt.imshow(Y_rec_np)
plt.title("Reconstruction")
plt.subplot(2,2,3)
plt.imshow(H_RGB)
plt.title("Hematoxylin")
plt.subplot(2,2,4)
plt.imshow(E_RGB)
plt.title("Eosin")
#plt.savefig(SAVE_FIG_PATH + f"camelyon_{idx}idx.pdf", bbox_inches='tight', pad_inches=0)
plt.show()

# WSSB

In [ ]:
organ = "Colon"
dataset = WSSBDatasetTest("/data/BasesDeDatos/Alsubaie/Data/", organ_list=[organ])
idx = 2
img, od_img, mR, C_gt, M_gt = dataset[idx]
od_img = od_img.to(DEVICE)
out_Mnet_mean, out_Mnet_var, out_Cnet, Y_rec = model.forward(od_img.unsqueeze(0))

In [ ]:
img_np = img.numpy().astype(np.uint).transpose(1,2,0)

Y_rec_np =  Y_rec.cpu().detach().numpy().squeeze()
Y_rec_np = od2rgb_np(undo_normalization(Y_rec_np))
Y_rec_np = np.clip(Y_rec_np, 0, 255).astype(np.uint8)
Y_rec_np = Y_rec_np.transpose(1,2,0)

In [ ]:
C_GT_OD = C_to_OD_torch(C_gt, M_gt).cpu().detach().numpy().squeeze()
H_OD_GT =  C_GT_OD[0, :, :]
H_RGB_GT = od2rgb_np(H_OD_GT).transpose(1,2,0)
H_RGB_GT = np.clip(H_RGB_GT, 0, 255).astype(np.uint8)
E_OD_GT =  C_GT_OD[1, :, :]
E_RGB_GT = od2rgb_np(E_OD_GT).transpose(1,2,0)
E_RGB_GT = np.clip(E_RGB_GT, 0, 255).astype(np.uint8)

C_OD = C_to_OD_torch(out_Cnet, out_Mnet_mean).cpu().detach().numpy().squeeze()

H_OD = C_OD[0, :, :, :]
#H_OD = np.clip(H_OD, 0.0, H_OD.max())
H_RGB = od2rgb_np(undo_normalization(H_OD))
H_RGB = (255.0*normalize_to1(H_RGB, max_val=H_RGB.max(), min_val = H_RGB.min())).astype(np.uint8)
#H_RGB = np.clip(H_RGB, 0, 255).astype(np.uint8)
H_RGB = H_RGB.astype(np.uint8)
H_RGB = H_RGB.transpose(1,2,0)

E_OD = C_OD[1, :, :, :]
#E_OD = np.clip(E_OD, 0.0, E_OD.max())
E_RGB = od2rgb_np(undo_normalization(E_OD))
E_RGB = (255.0*normalize_to1(E_RGB, max_val=E_RGB.max(axis=(0,1)), min_val = E_RGB.min(axis=(0,1))))
#E_RGB = np.clip(E_RGB, 0, 255)
E_RGB = E_RGB.astype(np.uint8)
E_RGB = E_RGB.transpose(1,2,0)

In [ ]:
print(C_GT_OD.min(), C_GT_OD.max())

In [ ]:
E_RGB.shape

In [ ]:
E_RGB.max(axis=(0,1)).shape

In [ ]:
plt.figure(figsize=(10,15))
plt.subplot(3,2,1)
plt.imshow(img_np)
plt.xlabel("Original")
plt.subplot(3,2,2)
plt.imshow(Y_rec_np)
plt.xlabel("Reconstruction")
plt.subplot(3,2,3)
plt.imshow(H_RGB_GT)
plt.xlabel("H GT")
plt.subplot(3,2,4)
plt.imshow(H_RGB)
plt.xlabel("H")
plt.subplot(3,2,5)
plt.imshow(E_RGB_GT)
plt.xlabel("E GT")
plt.subplot(3,2,6)
plt.imshow(E_RGB)
plt.xlabel("E")
#plt.savefig(SAVE_FIG_PATH + f"wssb_{organ}_{idx}idx.pdf", bbox_inches='tight', pad_inches=0)

plt.show()